In [10]:
from functions0944 import g_D_symbolic_coefficients_dict
from functions0944 import restore_matrices
from functions0944 import Augmented_Lagrangian
from functions0944 import update_Lagrangian_coefficients
from functions0944 import generate_M_d
import numpy as np
from scipy.integrate import quad
from functools import partial
import jax
import jax.numpy as jnp
from scipy.optimize import minimize

In [11]:
D = 5
d = 4
L = 2*d+1
rho = 10

In [12]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
1.6*x1**4 + x1**3/125 + 3*x1**2*x2/125 + 3*x1**2*x3/125 + 3*x1**2*x4/125 + 3*x1**2*x5/125 - 1.6*x1**2 + 3*x1*x2**2/125 + 6*x1*x2*x3/125 + 6*x1*x2*x4/125 + 6*x1*x2*x5/125 + 3*x1*x3**2/125 + 6*x1*x3*x4/125 + 6*x1*x3*x5/125 + 3*x1*x4**2/125 + 6*x1*x4*x5/125 + 3*x1*x5**2/125 + 1.6*x2**4 + x2**3/125 + 3*x2**2*x3/125 + 3*x2**2*x4/125 + 3*x2**2*x5/125 - 1.6*x2**2 + 3*x2*x3**2/125 + 6*x2*x3*x4/125 + 6*x2*x3*x5/125 + 3*x2*x4**2/125 + 6*x2*x4*x5/125 + 3*x2*x5**2/125 + 1.6*x3**4 + x3**3/125 + 3*x3**2*x4/125 + 3*x3**2*x5/125 - 1.6*x3**2 + 3*x3*x4**2/125 + 6*x3*x4*x5/125 + 3*x3*x5**2/125 + 1.6*x4**4 + x4**3/125 + 3*x4**2*x5/125 - 1.6*x4**2 + 3*x4*x5**2/125 + 1.6*x5**4 + x5**3/125 - 1.6*x5**2 + 1.0

Coefficients Dictionary:
Coefficient of (4, 0, 0, 0, 0): 1.6
Coefficient of (3, 0, 0, 0, 0): 0.008
Coefficient of (2, 1, 0, 0, 0): 0.024
Coefficient of (2, 0, 1, 0, 0): 0.024
Coefficient of (2, 0, 0, 1, 0): 0.024
Coefficient of (2, 0, 0, 0, 1): 0.024
Coefficient of (2, 0, 0, 0, 0): -1.6


In [13]:
# This block is for initialization according to the article, µ, R_i^l,is initialized randomly using a distribution U ([−1, 1]) uniform over [−1, 1].

# def function of the moments in M_d
def f(x,n):
    return 1/2*x**n

In [14]:
# This block is for the matrix initialization and flatten as x_input

x_mu_D_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]

# 9/28 update, we first generate a list of different moments of measure, then in the augumented lagrangian, we can generate the M_d matrix from our list of moments of measure
# This can fix the problem that when we try to differentiate the ϕn(µ), it actully differentiate the list of moments of measure not the M_d matrix 

# Define the mu_list
list_size = 2*d+3
my_list = np.array([quad(f,-1,1,args=(i))[0] for i in range(2*d+3)])

# generate the list of list of both matrix
for l in range(L):
    for i in range(D):
        x_mu_D_L_list[i].append(my_list)
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))

x_matrix_list_new = x_mu_D_L_list+x_R_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))
print(len(x_input))


1620


In [15]:
# Initialize the lagrangian coefficient as 0
Lagrangian_coefficient = 0
rho = 50

In [16]:
#implement the augumented lagrangian
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)

#use jax autogradient to generate the gradient
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)

In [17]:
print(aug_lagrangian_partial.keywords['Lagrangian_coefficient'])
print(aug_lagrangian_partial.keywords['rho'])

0
50


In [18]:
iteration = 0
while True:
    iteration += 1
    aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
    aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)

    result = minimize(aug_lagrangian_partial, x0=x_input,
                    method='L-BFGS-B',
                    jac=aug_lagrangian_partial_gradient,
                    options={
                        'gtol': 1e-2,             # Stopping criterion (relative gradient)
                        'ftol': 1e-5,             # Stopping criterion (absolute value)
                    })

    print("-"*40)
    print("This is {} iteration".format(iteration))

    print("Minimum value of the Augmented Lagrangian function:", result.fun)
    print("Was the optimization successful?", result.success)
    print("Number of iterations:", result.nit)
    print(result.message)

    Lagrangian_coefficient = update_Lagrangian_coefficients(d,D,L,result.x,Lagrangian_coefficient,rho)

    rho = abs(Lagrangian_coefficient)*25

    print("-"*40)

    x_input = result.x
    print("Lagrangian_coefficient:",Lagrangian_coefficient)
    print("rho:",rho)
    
    x_mu_D_L_list,_= restore_matrices(x_input,d,D,L)
    x_M_D_L_list = generate_M_d(x_mu_D_L_list,d,D,L)
    l_product_list = []
    for l in range(L):
        moment_product = 1
        for i in range(D):
            moment_product *= x_M_D_L_list[i][l][0,0]
        l_product_list.append(moment_product)

    max_index = l_product_list.index(max(l_product_list))

    Denominator_product = 1
    for i in range(D):
        Denominator_product *=  x_M_D_L_list[i][max_index][0,0]

    x_output = jnp.array([x_M_D_L_list[i][max_index][0,1]/Denominator_product  for i in range(D)])
    polynomial_values = polynomial(*x_output)
    x_exact = jnp.full((D,), -0.75553)
    print("relative error is {}".format(abs((polynomial_values+1.391)/1.391)))
    print("-"*40)
    if abs((polynomial_values+1.391)/1.391)<1e-2:
        break
    # if abs((polynomial_values+1.391)/1.391)<1e-2:
    #     break
    

print("The final point: ",x_output)
print("Objective function value:", polynomial_values)
print(iteration)


----------------------------------------
This is 1 iteration
Minimum value of the Augmented Lagrangian function: -15.166654586791992
Was the optimization successful? True
Number of iterations: 253
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------------------------
Lagrangian_coefficient: 47.194381242158876
rho: 1179.859531053972
relative error is 0.038854341953992844
----------------------------------------
----------------------------------------
This is 2 iteration
Minimum value of the Augmented Lagrangian function: -1107.58642578125
Was the optimization successful? True
Number of iterations: 38
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------------------------
Lagrangian_coefficient: -54701.83621449325
rho: 1367545.905362331
relative error is 0.5654628872871399
----------------------------------------
----------------------------------------
This is 3 iteration
Minimum value of the Augmented Lagrangian function: -1319332.875
Was the optimiz

KeyboardInterrupt: 

In [9]:
print("The final point: ",x_output)
print("Objective function value:", polynomial_values)
print(iteration)

NameError: name 'x_output' is not defined